In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import sqlite3
import statistics
soccer = sqlite3.connect("../input/soccer/database.sqlite")
P_A=pd.read_sql_query("SELECT * from Player_Attributes", soccer)
Player=pd.read_sql_query("SELECT * from Player", soccer)

In [ ]:
def infoPlayer(P_A_id):
    infoP_A=P_A.loc[(P_A.id == P_A_id)]
    infoPlay=Player.loc[(Player.player_api_id == int(infoP_A.player_api_id))]
    return str(infoPlay.player_name) + str(infoP_A.date) + str(infoPlay.player_api_id)
    
def getName(P_A_id):
    infoP_A=P_A.loc[(P_A.id == P_A_id)]
    infoPlay=Player.loc[(Player.player_api_id == int(infoP_A.player_api_id))]
    return infoPlay.player_name

def infoPlayer(row):
    P_a_i=int(row.player_api_id)
    infoPlay=Player[Player.player_api_id == P_a_i]
    return pd.DataFrame({'Name': infoPlay.player_name, 'Date': row.date, 'player_api_id': row.player_api_id})

def getName(row):
    P_a_i=int(row.player_api_id)
    return Player[Player.player_api_id == P_a_i].player_name

In [ ]:
def newskills(data):
    df=data
    df['attacking']= (df.crossing+df.finishing+df.heading_accuracy+df.short_passing+df.volleys)/5
    df['skill']= (df.dribbling+df.curve+df.free_kick_accuracy+df.long_passing+df.ball_control)/5
    df['movement']= (df.acceleration+df.sprint_speed+df.agility+df.reactions+df.balance)/5
    df['power']= (df.shot_power+df.jumping+df.stamina+df.strength+df.long_shots)/5
    df['mentality']= (df.aggression+df.interceptions+df.positioning+df.vision+df.penalties+df.marking)/6
    df['defending']= (df.standing_tackle+df.sliding_tackle)/2
    df['goalkeeping']= (df.gk_diving+df.gk_handling+df.gk_kicking+df.positioning+df.gk_reflexes)/5
    return df

In [ ]:
data=newskills(P_A)
#data=P_A

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data.iloc[:, 3:].corr(), cmap="Greens")

plt.figure(figsize=(3,10))
sns.heatmap(data.iloc[:, 3:].corr().loc['overall_rating'].to_frame(), cmap="Greens")

plt.figure(figsize=(3,10))
sns.heatmap(data.iloc[:, 3:].corr().loc['potential'].to_frame(), cmap="Greens")

In [ ]:
def search_player(data, foot, attack, defens, skill):
    
    df= data.sort_values(by = skill, ascending = False)
    if foot != 'empty':
        df=df[df.preferred_foot==foot]
    if attack != 'empty':
        df=df[df.attacking_work_rate==attack]
    if defens != 'empty':
        df=df[df.defensive_work_rate==defens]
    return df

In [ ]:
search_player(data, 'left', 'high', 'empty', 'crossing')

In [ ]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df



def create_base(dt, col):
    dt=dt.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date'], axis=1)
    dt=dt.drop(['preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1)
    dt = handle_non_numerical_data(dt).dropna()
    X=dt.drop(col, axis=1)
    Y=pd.DataFrame(dt[col])
    return X,Y

from sklearn.neighbors import KNeighborsClassifier



def learner(players, col):
    
    donnees=P_A
    
    data_X, data_Y = create_base(donnees, col)
    
    knn=KNeighborsClassifier(n_neighbors=1)
    knn.fit(data_X, data_Y)
    
    players=players.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date'], axis=1)
    players=players.drop(['preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1)
    players=players.dropna()
    
    y_pred=knn.predict(players)
    
    return y_pred

In [ ]:
players = pd.DataFrame({'id':[0], 'player_fifa_api_id':[0], 'player_api_id':[0], 'date':[0], 
                        'overall_rating': [91], 
                        'preferred_foot':['right'], 'attacking_work_rate':['high'], 'defensive_work_rate':['low'], 'crossing':[78], 'finishing':[93], 'heading_accuracy':[72],
       'short_passing':[85], 'volleys':[83], 'dribbling':[93], 'curve':[80], 'free_kick_accuracy':[69],
       'long_passing':[71], 'ball_control':[91], 'acceleration':[97], 'sprint_speed':[97],
       'agility':[92], 'reactions':[93], 'balance':[83], 'shot_power':[86], 'jumping':[78], 'stamina':[88],
       'strength':[77], 'long_shots':[82], 'aggression':[62], 'interceptions':[38], 'positioning':[92],
       'vision':[82], 'penalties':[79], 'marking':[88], 'standing_tackle':[34], 'sliding_tackle':[32],
       'gk_diving':[13], 'gk_handling':[5], 'gk_kicking':[7], 'gk_positioning':[11],
       'gk_reflexes':[6]})

Messi = {
    'overall_rating': 94, 
    'crossing': 80, 'finishing':93, 'heading_accuracy':71,
       'short_passing':88, 'volleys': 85, 'dribbling':96, 'curve':89, 'free_kick_accuracy':90,
       'long_passing':79, 'ball_control':96, 'acceleration':95, 'sprint_speed':90,
       'agility':92, 'reactions':92, 'balance':95, 'shot_power':80, 'jumping':68, 'stamina':75,
       'strength':59, 'long_shots':88, 'aggression':48, 'interceptions':22, 'positioning':90,
       'vision':90, 'penalties':74, 'marking':13, 'standing_tackle':23, 'sliding_tackle':21,
       'gk_diving':6, 'gk_handling':11, 'gk_kicking':15, 'gk_positioning':14,
       'gk_reflexes':8}

Ronaldo = {
    'overall_rating': 93, 
    'crossing': 82, 'finishing':95, 'heading_accuracy':86,
       'short_passing':81, 'volleys': 87, 'dribbling':93, 'curve':88, 'free_kick_accuracy':77,
       'long_passing':72, 'ball_control':91, 'acceleration':91, 'sprint_speed':93,
       'agility':90, 'reactions':92, 'balance':62, 'shot_power':94, 'jumping':94, 'stamina':90,
       'strength':79, 'long_shots':93, 'aggression':62, 'interceptions':29, 'positioning':93,
       'vision':81, 'penalties':85, 'marking':22, 'standing_tackle':31, 'sliding_tackle':23,
       'gk_diving':7, 'gk_handling':11, 'gk_kicking':15, 'gk_positioning':14,
       'gk_reflexes':11}

Griezmann = {
    'overall_rating': 84, 
    'crossing': 82, 'finishing':87, 'heading_accuracy':78,
       'short_passing':77, 'volleys': 83, 'dribbling':86, 'curve':84, 'free_kick_accuracy':74,
       'long_passing':75, 'ball_control':84, 'acceleration':86, 'sprint_speed':86,
       'agility':89, 'reactions':83, 'balance':70, 'shot_power':79, 'jumping':83, 'stamina':76,
       'strength':62, 'long_shots':78, 'aggression':69, 'interceptions':35, 'positioning':88,
       'vision':74, 'penalties':71, 'marking':23, 'standing_tackle':22, 'sliding_tackle':11,
       'gk_diving':14, 'gk_handling':8, 'gk_kicking':14, 'gk_positioning':13,
       'gk_reflexes':14}

players = players.append(Messi, ignore_index = True)
players = players.append(Ronaldo, ignore_index = True)
players = players.append(Griezmann, ignore_index = True)

In [ ]:
learner(players, 'potential')